In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [142]:
custom_template = {
    "layout": go.Layout(
        plot_bgcolor="#fafafa",
        paper_bgcolor="#ffffff",
        colorway=px.colors.qualitative.G10,
    )
}

In [143]:
df = pd.read_csv("../data/BeerStyleMinMaxMean.csv")

In [11]:
for col in df.columns:
    print(col)

BeerStyle
Ca2+(mg/L)_min
Ca2+(mg/L)_max
Ca2+(mg/L)_mean
Mg2+(mg/L)_min
Mg2+(mg/L)_max
Mg2+(mg/L)
Na+(mg/L)_min
Na+(mg/L)_max
Na+(mg/L)_mean
Cl(mg/L)_min
Cl(mg/L)_max
Cl(mg/L)_mean
SO42(mg/L)_min
SO42(mg/L)_max
SO42(mg/L)_mean
HCO3(mg/L)_min
HCO3(mg/L)_max
HCO3(mg/L)_mean
pH_min
pH_max
pH_mean
TDS(mg/L)_min
TDS(mg/L)_max
TDS(mg/L)_mean


In [63]:
for style in df["BeerStyle"].unique():
    print(style)

Pilsner(Czech)
BritishPaleAle
IrishStout
Gose
AmericanPaleAle
AmericanIPA
ImperialIPA(IIPA)
GermanHefeweizen
AmericanStout
RussianImperialStout
AmericanLager
ViennaLager
MunichHelles
AmericanAmberAle
BelgianTripel
Saison
GermanPilsner
Kölsch
BalticPorter
EnglishBarleywine


In [106]:
mean_cols = [x for x in df.columns if "mean" in x and x not in ("pH_mean", "TDS(mg/L)_mean")] + ["BeerStyle"]
print(mean_cols)
df_means = df[mean_cols]

['Ca2+(mg/L)_mean', 'Na+(mg/L)_mean', 'Cl(mg/L)_mean', 'SO42(mg/L)_mean', 'HCO3(mg/L)_mean', 'BeerStyle']


In [107]:
min_cols = [x for x in df.columns if "min" in x and x not in ("pH_min", "TDS(mg/L)_min")] + ["BeerStyle"]
print(min_cols)
df_mins = df[min_cols]

['Ca2+(mg/L)_min', 'Mg2+(mg/L)_min', 'Na+(mg/L)_min', 'Cl(mg/L)_min', 'SO42(mg/L)_min', 'HCO3(mg/L)_min', 'BeerStyle']


In [108]:
max_cols = [x for x in df.columns if "max" in x and x not in ("pH_max", "TDS(mg/L)_max")] + ["BeerStyle"]
print(max_cols)
df_maxs = df[max_cols]

['Ca2+(mg/L)_max', 'Mg2+(mg/L)_max', 'Na+(mg/L)_max', 'Cl(mg/L)_max', 'SO42(mg/L)_max', 'HCO3(mg/L)_max', 'BeerStyle']


In [86]:
df_means.head()

,Ca2+(mg/L)_mean,Na+(mg/L)_mean,Cl(mg/L)_mean,SO42(mg/L)_mean,HCO3(mg/L)_mean,BeerStyle
0,30,25,25,25,75,Pilsner(Czech)
1,100,40,45,250,175,BritishPaleAle
2,100,40,45,100,275,IrishStout
3,100,225,225,25,125,Gose
4,100,25,25,250,125,AmericanPaleAle


In [146]:
def plot_polar(df, category_column="BeerStyle", categories_allowed=[]):
    categories = list(df[category_column])
    
    fig = go.Figure(layout=dict(width=1200, height=800))

    for cat in categories:
        # skip this if its not in categories
        if categories_allowed and cat not in categories_allowed:
            continue
            
        # get the rof for this category
        row = df[df[category_column] == cat]

        # drop the category column
        row = row.drop(category_column, axis=1)

        # get our values
        theta = [x.replace("_mean", "") for x in row.columns]        
        r = row.values[0]
   
        fig.add_trace(go.Scatterpolar(
            r=r,
            theta=theta,
            fill='toself',
            name=cat,
            connectgaps=True,
        ))

        
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
              visible=False,
            )
        ),
        showlegend=True,
        title_text="Tonights Water Profiles Compared",
        title_x=0.5,
        template=custom_template,
    )
        
    fig.show()

In [147]:
plot_polar(df_means, categories_allowed=["Kölsch", "BritishPaleAle", "Gose"])

In [196]:
def plot_style_min_max(beer_style: str, df_min=df_mins, df_max=df_maxs):
    fig = go.Figure(layout=dict(width=800, height=800))

    # get the rof for this category
    row_min = df_min[df_min["BeerStyle"] == beer_style]

    # drop the category column
    row_min = row_min.drop("BeerStyle", axis=1)

    # get our values
    theta_min = [x.replace("_min", "") for x in row_min.columns]
    theta_min += [theta_min[0]]

    r_min = list(row_min.values[0])
    r_min += [r_min[0]]
    
    # add the min
    fig.add_trace(go.Scatterpolar(
        r=r_min,
        theta=theta_min,
        fill=None,
        name="Min",
        connectgaps=True,
        marker={"color": px.colors.qualitative.G10[2]}
    ))

    # get the rof for this category
    row_max = df_max[df_max["BeerStyle"] == beer_style]

    # drop the category column
    row_max = row_max.drop("BeerStyle", axis=1)

    # get our values
    theta_max = [x.replace("_max", "") for x in row_max.columns]
    theta_max += [theta_max[0]]
    r_max = list(row_max.values[0])
    r_max += [r_max[0]]
    
    # add the max
    fig.add_trace(go.Scatterpolar(
        r=r_max,
        theta=theta_max,
        fill='tonext',
        name="Max",
        connectgaps=True,
        marker={"color": px.colors.qualitative.G10[2]}
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
              visible=True,
              gridcolor="#eee"
            )
        ),
        showlegend=False,
        title_text=f"{beer_style}<br><span style='font-size: 14px;color: #aaa;'>minimum and maximum allowable values for style</span>",
        title_x=0.5,
        template=custom_template,
    )
        
    fig.show()

In [197]:
for name in ["Kölsch", "BritishPaleAle", "Gose"]:
    plot_style_min_max(name)